In [1]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks

from c3.libraries import fidelities
from c3.parametermap import ParameterMap
from utils import *
from c3.experiment import Experiment
import c3.libraries.constants as constants
import c3.utils.qt_utils as qt_utils

2021-08-23 14:45:14.723824: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-23 14:45:14.723856: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def findPeaks(x, y, N: int) -> Tuple:
    """
    Returns the x and y values of the N largest local maxima in y.
    """
    peaks = find_peaks(y)[0]
    peakX = x[peaks]
    peakY = y[peaks]
    maxIndices = peakY.argsort()[-N:][::-1]
    return peakX[maxIndices], peakY[maxIndices]


# plotting functions
def plotSignal(time, signal, filename=None, spectrum_cut=1e-4) -> None:
    """
    Plots a time dependent signal and its normalised frequency spectrum.

    Parameters
    ----------
    time
        timestamps
    signal
        signal value
    filename: str
        Optional name of the file to which the plot will be saved. If none,
        it will only be shown.
    spectrum_cut:
        If not None, only the part of the normalised spectrum will be plotted
        whose absolute square is larger than this value.

    Returns
    -------

    """
    # plot time domain
    time = time.flatten()
    signal = signal.flatten()
    print("time domain: ", time.shape, signal.shape)
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[0].set_title('Signal')
    axs[0].plot(time, signal)
    axs[0].set_xlabel('time')

    # calculate frequency spectrum
    freq_signal = np.fft.rfft(signal)
    if np.abs(np.max(freq_signal)) > 1e-14:
        normalised = freq_signal / np.max(freq_signal)
    else:
        normalised = freq_signal
    freq = np.fft.rfftfreq(len(time), time[-1] / len(time))
    print("frequency domain: ", freq.shape, freq_signal.shape)

    peakFrequencies,peakValues = findPeaks(freq, np.abs(normalised) ** 2, 2)
    print("peaks: ", np.sort(peakFrequencies))

    # cut spectrum if necessary
    if spectrum_cut is not None:
        limits = np.flatnonzero(np.abs(normalised) ** 2 > spectrum_cut)
        freq = freq[limits[0]:limits[-1]]
        normalised = normalised[limits[0]:limits[-1]]

    # plot frequency domain
    axs[1].set_title('Spectrum')
    axs[1].plot(freq, normalised.real, label="Re")
    axs[1].plot(freq, normalised.imag, label="Im")
    axs[1].plot(freq, np.abs(normalised) ** 2, label="Square")
    axs[1].set_xlabel('frequency')
    axs[1].legend()

    # show and save
    plt.tight_layout()
    if filename:
        print("saving plot in " + filename)
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()


def plotOccupations(
        experiment: Experiment,
        populations: np.array,
        gate_sequence: List[str],
        level_names: List[str] = None,
        filename: str = None,
) -> None:
    """
    Plots time dependent populations. They need to be calculated with `runTimeEvolution` first.

    Parameters
    ----------
    experiment: Experiment
        The experiment containing the model and propagators
    populations: np.array
        Population vector for each time step
    gate_sequence: List[str]
        List of gate names that will be applied to the state
    level_names: List[str]
        Optional list of names for the levels. If none, the default list
        from the experiment will be used.
    filename: str
        Optional name of the file to which the plot will be saved. If none,
        it will only be shown.

    Returns
    -------

    """
    # plot populations
    fig, axs = plt.subplots(1, 1)
    dt = experiment.ts[1] - experiment.ts[0]
    ts = np.linspace(0.0, dt * populations.shape[1], populations.shape[1])
    axs.plot(ts / 1e-9, populations.T)

    # plot vertical lines
    gate_steps = [experiment.partial_propagators[g].shape[0] for g in gate_sequence]
    for i in range(1, len(gate_steps)):
        gate_steps[i] += gate_steps[i - 1]
    gate_times = gate_steps * dt
    plt.vlines(gate_times / 1e-9, tf.reduce_min(populations), tf.reduce_max(populations),
               linestyles=':', colors="black")

    # set plot properties
    axs.tick_params(direction="in", left=True, right=True, top=False, bottom=True)
    axs.set_xlabel('Time [ns]')
    axs.set_ylabel('Population')
    plt.legend(level_names if level_names else experiment.pmap.model.state_labels)
    plt.tight_layout()

    # show and save
    if filename:
        print("saving plot in " + filename)
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()


In [7]:
def createPulse(t_final: float, anharm: float, envelope: pulse.Envelope, file_suffix: str) -> None:
    # preparation
    active_levels = 4
    occupied_levels = [0, 2]
    directory = "./output"
    output_dir = pathlib.Path(directory)
    output_dir.mkdir(parents=True, exist_ok=True)

    # model
    q1 = createQubit(1, 5, 5e9, -anharm)
    model = createModel([q1])
    generator = createGenerator(model)

    # gate
    ideal = qt_utils.np_kron_n([
        constants.Id,
        constants.X,
    ])
    gate = createSingleQubitGate("lower-X", t_final, 4.7e9, envelope, model, q1, ideal)
    signal = generator.generate_signals(gate)[getDrive(model, q1).name]
    plotSignal(signal['ts'].numpy(), signal['values'].numpy(), directory + f"/signal_{file_suffix}.png",
               spectrum_cut=1e-4)


def runExperiment(
        t_final: float,
        freq: float,
        anharm: float,
        carrier_freq: float,
        envelope: pulse.Envelope,
        file_suffix: str
) -> None:
    # preparation
    active_levels = 4
    occupied_levels = [0, 2]
    directory = "./output"
    output_dir = pathlib.Path(directory)
    output_dir.mkdir(parents=True, exist_ok=True)

    # model
    q1 = createQubit(1, 5, freq, -anharm)
    model = createModel([q1])
    generator = createGenerator(model)
    energies = q1.get_Hamiltonian().numpy().diagonal().real
    print("energies: ", [(energies[i+1] - energies[i])/(2*np.pi) for i in range(len(energies)-1)])

    # gate
    #envelope = createPWCGaussianPulse(t_final, t_final / 4, 30)
    ideal = qt_utils.np_kron_n([
        constants.Id,
        constants.X,
    ])
    gate = createSingleQubitGate("lower-X", t_final, carrier_freq, envelope, model, q1, ideal)
    gates = [gate]
    gate_names = list(map(lambda g: g.get_key(), gates))

    # experiment
    exp = Experiment(pmap=ParameterMap(instructions=gates, model=model, generator=generator))
    exp.set_opt_gates(gate_names)
    unitaries = exp.compute_propagators()
    #print('unitaries: ', dict(map(lambda kv: (kv[0], kv[1].numpy().shape), unitaries.items())))

    # initial state
    init_state = createState(model, occupied_levels)
    #state = init_state.numpy().flatten()
    #print("initial state=", state, ", occupation=", exp.populations(state, model.lindbladian).numpy())

    # time evolution and signal before optimisation
    sequence = ["lower-X[0]"]
    populations = runTimeEvolutionDefault(exp, init_state, sequence)
    plotOccupations(exp, populations, sequence, filename=directory + f"/populations_before_{file_suffix}.png")
    signal = generator.generate_signals(gate)[getDrive(model, q1).name]
    plotSignal(signal['ts'].numpy(), signal['values'].numpy(), directory + f"/signal_before_{file_suffix}.png", spectrum_cut=1e-3)

    # add all optimisable parameters to a map
    drives = filterValues(model.couplings, chip.Drive)
    gateset_opt_map = []
    for gate in gates:
        for target in gate.targets:
            # TODO: target as index might not always work
            drive = drives[target]
            gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "amp")])
            #gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "freq_offset")])
            #gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "xy_angle")])
            #gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "delta")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "t_final")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "sigma")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "sigma2")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "gauss", "relative_amp")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "carrier", "freq")])
            gateset_opt_map.append([(gate.get_key(), drive.name, "carrier", "framechange")])

    # optimise
    optimisable_gates = list(filter(lambda g: g.get_key() != "id[]", gates))
    callback = lambda fidelity: print(fidelity)
    params_before, final_fidelity, params_after = optimise(
        exp, optimisable_gates,
        optimisable_parameters=gateset_opt_map,
        fidelity_fctn=fidelities.state_transfer_infid_set,
        fidelity_params={
            'psi_0': init_state[:active_levels],
            'active_levels': 4,
        },
        callback=callback,
        log_dir=(directory + ("/log_{0}_{1:.2f}/".format(file_suffix, t_final * 1e9)))
    )
    print('before:\n', params_before)
    print('after:\n', params_after)
    print('fidelity:\n', final_fidelity)

    # time evolution and signal after optimisation
    populations = runTimeEvolutionDefault(exp, init_state, sequence)
    plotOccupations(exp, populations, sequence, filename=directory + f"/populations_after_{file_suffix}.png")
    signal = generator.generate_signals(gate)[getDrive(model, q1).name]
    plotSignal(signal['ts'].numpy(), signal['values'].numpy(), directory + f"/signal_after_{file_suffix}.png", spectrum_cut=1e-3)


'''
directory = "./output"
output_dir = pathlib.Path(directory)
output_dir.mkdir(parents=True, exist_ok=True)

t_final = 7e-9
sigma_factors = np.arange(4, 200, 5)
for sigma_factor in sigma_factors:
    print(sigma_factor, t_final / sigma_factor)
    envelope = createPWCGaussianPulse(t_final, t_final / sigma_factor, 40)
    ts = np.linspace(0, t_final, 10000)
    vals = envelope.get_shape_values(ts)
    print(vals)
    plotSignal(ts, vals, directory + f"/envelope_{sigma_factor}.png")
'''

t_final = 10e-9
sigma = t_final / 11
sigma2 = sigma
relative_amp = np.sqrt(2)
freq = 5e9
anharm = 330e6
carrier_freq = 4.7e9
#for idx,delta in enumerate(np.linspace(-1, 3, 100)):
#    print(idx, delta)
#    createPulse(t_final, createGaussianPulse(t_final, sigma, delta), f"gaussian_{idx:02d}_delta{delta}")
#runExperiment(t_final, createPWCGaussianPulse(t_final, sigma, 30), "pwcgauss")
#createPulse(t_final, freq, anharm, createGaussianPulse(t_final, sigma, 6 * anharm), "gaussian")
runExperiment(t_final, freq, anharm, carrier_freq,
              createGaussianPulse(t_final, sigma, sigma2, relative_amp),
              "gaussian")

energies:  [5000000000.0, 4670000000.000003, 4339999999.999994, 4010000000.0000033]
saving plot in ./output/populations_before_gaussian.png
time domain:  (1000,) (1000,)
frequency domain:  (501,) (501,)
peaks:  [4.50225113e+09 5.00250125e+09]
saving plot in ./output/signal_before_gaussian.png
C3:STATUS:Saving as: /home/user/c3/output/log_gaussian_10.00/c1_state_transfer_infid_set_lbfgs/2021_08_23_T_14_51_10/open_loop.log
0.9829277376806347


/home/user/c3/c3/optimizers/optimizer.py:290: Warning: lower-X[0]-d1-carrier-framechange has no gradient. This might indicate no usage for current experiment.
  warnings.warn(
/home/user/c3/c3/signal/gates.py:260: UserWarning: Length of instruction lower-X[0] is fixed, but cuts at least one component. d1-gauss is should end @ 12.805 ns , but instruction ends at 1e-08
  warnings.warn(


before:
 lower-X[0]-d1-gauss-amp               : 500.000 mV 
lower-X[0]-d1-gauss-t_final           : 10.000 ns 
lower-X[0]-d1-gauss-sigma             : 909.091 ps 
lower-X[0]-d1-gauss-sigma2            : 909.091 ps 
lower-X[0]-d1-gauss-relative_amp      : 1.414  
lower-X[0]-d1-carrier-freq            : 4.750 GHz 2pi 
lower-X[0]-d1-carrier-framechange     : 0.000 rad 

after:
 0.9829277376806347
fidelity:
 lower-X[0]-d1-gauss-amp               : 500.000 mV 
lower-X[0]-d1-gauss-t_final           : 10.000 ns 
lower-X[0]-d1-gauss-sigma             : 909.091 ps 
lower-X[0]-d1-gauss-sigma2            : 909.091 ps 
lower-X[0]-d1-gauss-relative_amp      : 1.414  
lower-X[0]-d1-carrier-freq            : 4.750 GHz 2pi 
lower-X[0]-d1-carrier-framechange     : 0.000 rad 

saving plot in ./output/populations_after_gaussian.png
time domain:  (1000,) (1000,)
frequency domain:  (501,) (501,)
peaks:  [4.50225113e+09 5.00250125e+09]
saving plot in ./output/signal_after_gaussian.png
